In [8]:
import scipy.io as sio
import numpy as np 
import sys 

#145 145 220
def main():
    # See https://rslab.ut.ac.ir/data for information about this dataset (Urban)
    hs_image = sio.loadmat("data/Urban_R162.mat")
    # print(type(hs_image))

    # for key in hs_image:
    #     print(key)

    # Channels that are kept 
    # print(hs_image["SlectBands"])

    # Size of image 
    # print(hs_image["nRow"])
    # print(hs_image["nCol"])

    # Number of wavelengths 
    # print(hs_image["nBand"])

    # Matrix itself 
    # print(hs_image["Y"])

    # Not sure what this mean 
    # print(hs_image["maxValue"])

    
    matrix = np.array(hs_image["Y"])

    # Each row correspond to a channel/wavelength and the image is vectorized as a row vector (I am not sure how it is being flatten) 
    # I am not sure what the value in the matrix mean. They range from 0 to 1000. I am guessing it is how much is captured by a particular channel after normalization  
    print(matrix.shape)

    

main()

(162, 94249)
